In [1]:
from __future__ import division

In [ ]:
# For cpu only
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [2]:
import tensorflow as tf
import tflearn

In [3]:
import numpy as np

In [4]:
from tensorflow.python import debug as tf_debug

In [5]:
sess = tf.Session()
# sess = tf_debug.LocalCLIDebugWrapperSession(sess)
lr = tf.placeholder(tf.float32, shape=[])
optim = tf.train.AdamOptimizer(learning_rate=lr)
# sgd_optim = tf.train.GradientDescentOptimizer(learning_rate=lr)

In [6]:
# sess.add_tensor_filter("has_inf_or_nan", tf_debug.has_inf_or_nan)

In [7]:
from tqdm import tqdm

In [8]:
from bokeh.plotting import figure, output_notebook, show
output_notebook()

Loading BokehJS ...

In [9]:
# import BayesianNN

In [10]:
# bnn = BayesianNN.BayesianFC([4,32,32,1])

In [11]:
import Bayesian_FC_Layer, Bayesian_Net

In [12]:
def bbnet(i, o, mean_init, std_init, prior_mean, prior_std, activation=tf.nn.relu):
    return Bayesian_FC_Layer.Bayesian_FC(i, o, 
                                         qw_mean_initial=mean_init, 
                                         qw_std_initial=std_init, 
                                         qb_std_initial=std_init,
                                         pw_mean=prior_mean,
                                         pw_sigma=prior_std,
                                         pb_mean=prior_mean,
                                         pb_sigma=prior_std,
                                         activation=activation
                                         )

In [13]:
std_init = 0.1
mean_init = 0.1
prior_mean = 0.0
prior_std = 1.0
l1 = bbnet(4, 32, mean_init, std_init, prior_mean, prior_std)
l2 = bbnet(32, 32, mean_init, std_init, prior_mean, prior_std)
l3 = bbnet(32, 1, mean_init, std_init, prior_mean, prior_std, activation=tf.identity)

def bnet_model(input_tensor, local_reparam_trick=False):
    net = l1.sample(input_tensor, local_reparam_trick)
    net = l2.sample(net, local_reparam_trick)
    net_out = l3.sample(net, local_reparam_trick)
    return net_out
# l1 = Bayesian_FC_Layer.Bayesian_FC(4, 32, qw_mean_initial=mean_init, qw_std_initial=std_init, qb_std_initial=std_init)
# l2 = Bayesian_FC_Layer.Bayesian_FC(32, 32, qw_mean_initial=mean_init, qw_std_initial=std_init, qb_std_initial=std_init)
# l3 = Bayesian_FC_Layer.Bayesian_FC(32, 32, qw_mean_initial=mean_init, qw_std_initial=std_init, qb_std_initial=std_init)
# l4 = Bayesian_FC_Layer.Bayesian_FC(32, 32, qw_mean_initial=mean_init, qw_std_initial=std_init, qb_std_initial=std_init)
# l5 = Bayesian_FC_Layer.Bayesian_FC(32, 1, activation=tf.identity, qw_mean_initial=mean_init, qw_std_initial=std_init, qb_std_initial=std_init)

In [14]:
from Bayesian_Net import log_gaussian_pdf

In [15]:
def bnet_prob(pred, tar):
    return log_gaussian_pdf(pred, tar, 0.1)

In [16]:
bnet = Bayesian_Net.Bayesian_Net([l1, l2, l3], bnet_prob)

In [17]:
def test_func(x):
    e = np.random.normal(0,0.01)
    return x + 0.3*np.sin(2 *np.pi * (x + e)) + 0.3*np.sin(4 * np.pi*(x + e)) + e

In [18]:
data = [(x, test_func(x)) for x in np.linspace(0,0.5,1000)]

In [19]:
data_input = list(map(lambda x: [x[0]], data))
data_input_poly = list(map(lambda x: [x[0], x[0]**2, x[0]**3, x[0]**4], data))
data_target = list(map(lambda x: [x[1]], data))

In [20]:
def normal_nn(optim):
    data_input = tf.placeholder(tf.float32, shape=[None, 1])
    data_target = tf.placeholder(tf.float32, shape=[None, 1])
    
    poly = tf.concat(1, [data_input, data_input**2, data_input**3, data_input**4])
    net = tflearn.fully_connected(poly, 32, activation="relu")
#     net = tflearn.fully_connected(net, 32, activation="relu")
#     net = tflearn.fully_connected(net, 32, activation="relu")
#     net = tflearn.fully_connected(net, 32, activation="relu")
#     net = tflearn.fully_connected(net, 32, activation="relu")
    
    data_output = tflearn.fully_connected(net, 1, activation="linear")
    
    loss = tf.reduce_mean(tf.square(data_output - data_target))
    op = optim.minimize(loss)
    
    return op, data_input, data_target, loss, data_output

In [21]:
batch_size = 128

In [22]:
epoch_loss = []
nn_epoch_loss = []

In [23]:
epochs = 2000 

In [24]:
test_set = [[x] for x in np.linspace(-1,1,1000)]
test_set_poly = [[x,x**2,x**3,x**4] for x in np.linspace(-1,1,1000)]
test_set_flat = [a[0] for a in test_set]
data_input_flat = [a[0] for a in data_input]
data_target_flat = [a[0] for a in data_target]

In [25]:
op, data_x, data_y, loss, data_output = normal_nn(optim)
sess.run(tf.global_variables_initializer())
for e in tqdm(range(1000 * 2)):
    indices = np.random.randint(low=0, high=1000, size=batch_size)
    input_batch = [data_input[i] for i in indices]
    target_batch = [data_target[i] for i in indices]

    _, l = sess.run([op, loss], feed_dict={data_x: input_batch, data_y:target_batch, lr:0.001})
    nn_epoch_loss.append(l)

100%|██████████| 2000/2000 [00:01<00:00, 1158.37it/s]


In [26]:
nn_preds = sess.run(data_output, feed_dict={data_x: test_set})

In [27]:
# op, data_x, data_y, m_scaling, b_size, loss, kl_loss, data_loss, grads = bnn.update_v2(8, optim)
# sess.run(tf.global_variables_initializer())

In [28]:
bnn_data_input = tf.placeholder(tf.float32, shape=[None, 4])
bnn_data_target = tf.placeholder(tf.float32, shape=[None, 1])
kls = tf.placeholder(tf.float32, shape=[])

bnet_loss, klloss, dataloss = bnet.loss(bnn_data_input, bnet_model, bnn_data_target,kl_scaling=kls, N=8)
bnet_output = bnet_model(bnn_data_input, local_reparam_trick=False)

copy_old = bnet.copy_variational_parameters()
kl_div = bnet.kl_new_and_old()

grads_to_apply = optim.compute_gradients(bnet_loss)
# print(grads_to_apply)

clipped_policy_grads = []
for (grad, var) in grads_to_apply:
    if grad is not None:
        clipped_policy_grads.append((tf.clip_by_norm(grad, 10), var))

# clipped_policy_grads = grads_to_apply
grad_op = optim.apply_gradients(clipped_policy_grads)

sess.run(tf.global_variables_initializer())

In [29]:
bnn_lr = 0.001

In [30]:
epoch_loss=[]
kl_loss=[]
data_loss=[]
kl_divs=[]

In [31]:
for e in tqdm(range(epochs*10)):
#     input_batch = data_input_poly
#     target_batch = data_target
    if e > epochs*10*0.5:
        hh = 1000
    else:
        hh = 500
    indices = np.random.randint(low=0, high=hh, size=batch_size)
    input_batch = [data_input_poly[i] for i in indices]
    target_batch = [data_target[i] for i in indices]
    
#     kl = 0.0
    kl = batch_size/1000
#     kl = 1.0
#     kl = (2**(epochs - e - 1))/( (2**epochs) - 1) 

#     kl = 0.0

    _, l, kloss, dloss = sess.run([grad_op, bnet_loss, klloss, dataloss], feed_dict={kls:kl, bnn_data_input: input_batch, bnn_data_target:target_batch, lr:bnn_lr})
    epoch_loss.append(l)
    
    if e % 50 == 0:
        div = sess.run(kl_div)
        sess.run(copy_old)
        kl_divs.append(div)
    
    if e % 5 == 0:
        epoch_loss.append(l) 
        kl_loss.append(kloss)
        data_loss.append(dloss)

100%|██████████| 20000/20000 [02:15<00:00, 147.56it/s]


In [32]:
sess.run(bnet.layers[0].qw_sigma)

array([[ 0.5547753 ,  0.3238045 ,  0.01497539,  0.22829375,  0.53043091,
         0.54480422,  0.5199998 ,  0.565624  ,  0.55838829,  0.56214678,
         0.55327874,  0.53741801,  0.40567228,  0.00850614,  0.55970973,
         0.57125956,  0.5145793 ,  0.49415186,  0.55879092,  0.57435524,
         0.00620688,  0.54254121,  0.00569993,  0.52963191,  0.55371052,
         0.54592752,  0.53618312,  0.54592019,  0.54193664,  0.5020318 ,
         0.56051743,  0.55416805],
       [ 0.82732964,  0.61700583,  0.08106108,  0.48462844,  0.81628591,
         0.82024717,  0.79708815,  0.83972502,  0.829723  ,  0.82858264,
         0.82719302,  0.82259876,  0.71476644,  0.03755207,  0.83052838,
         0.83930504,  0.80391932,  0.79507709,  0.8290056 ,  0.84872127,
         0.02205057,  0.82059044,  0.01466307,  0.81433505,  0.82575047,
         0.82327521,  0.81274205,  0.81670696,  0.8228755 ,  0.79939592,
         0.8329252 ,  0.82606733],
       [ 0.95727706,  0.88632375,  0.37185162,  0.7938

In [33]:
# for e in tqdm(range(epochs)):
# #     indices = np.random.randint(low=0, high=1000, size=batch_size)
# #     input_batch = [data_input_poly[i] for i in indices]
# #     target_batch = [data_target[i] for i in indices]
#     input_batch = data_input_poly
#     target_batch = data_target
    
# #     ms = (2**(epochs - e - 1))/( (2**epochs) - 1) 
# #     ms = 1000/(batch_size)
# #     ms = (1000*epochs)/(batch_size*(e+1))
# #     ms = (1000)/(batch_size + e)
#     ms = 0
# #     ms = 1.0
    
# #     grad_nums = [a for (a,b) in grads]
#     qw_mean, qw_p, qb_mean, qb_p = sess.run([bnn.qw_means[0], bnn.qw_ps[0], bnn.qb_means[0], bnn.qb_ps[0]])
#     _, l, k_loss, d_loss, gg = sess.run([op, loss, kl_loss, data_loss, grads], feed_dict={data_x: input_batch, data_y:target_batch, m_scaling:ms, b_size:batch_size, lr:bnn_lr})
# #     print("----------Epoch {}-----------".format(e))
# #     print("{}\n{}\n".format(qb_mean, qb_p))
# #     print(gg[2:])
# #     print()
# #     print("KL:{}, Data:{}\n".format(k_loss, d_loss))
#     if e % 5 == 0:
#         epoch_loss.append(l)   

In [34]:
# sess.run(bnn.qw_ps[0])

In [35]:
# sess.run(bnn.qw_ps[0])
# grads = tf.gradients(loss, w_vars)
# sess.run(w_vars, feed_dict={data_x: data_input_poly, data_y:target_batch, m_scaling:[0.1]})

In [36]:
# pred_input = tf.placeholder(tf.float32, shape=[None, 4])
# pred_output = bnn.sampled_output(pred_input, 1000)
# pred_input, pred_output = bnn.sample(use_mean=False)
predictions = sess.run(bnet_output, feed_dict={bnn_data_input: test_set_poly})
var_trials = 500
for _ in tqdm(range(var_trials-1)):
    predictions = np.concatenate([predictions, sess.run(bnet_output, feed_dict={bnn_data_input: test_set_poly})], axis=1)

100%|██████████| 499/499 [00:00<00:00, 1045.27it/s]


In [37]:
pred_means = np.apply_along_axis(lambda x: np.mean(x), 1, predictions)
pred_means[3]

1.1493294

In [38]:
pred_vars = np.apply_along_axis(lambda x: np.std(x, ddof=1), 1, predictions)

In [39]:
predictions.shape

(1000, 500)

In [40]:
p = figure(width=800, height=800, y_range=(-0.5,1.5), x_range=(-1,1))

p.line(data_input_flat, data_target_flat, line_width=0.5, color=(0,0,255,0.5))

err_xs=[]
err_ys=[]
for x, y, yerr in zip(test_set_flat, pred_means, pred_vars):
    err_xs.append((x, x))
    err_ys.append((y - yerr, y + yerr))

p.multi_line(err_xs, err_ys, color=(255,0,0,0.15))

# p.line([a/100 for a in range(100)], [scale*a/100 for a in range(100)], line_width=3, color="blue")
p.line(test_set_flat, pred_means, line_width=2, color="red")

for i in range(0, 200, 25):
    pp = list(map(lambda x: x[i], predictions))
    p.line(test_set_flat, pp, line_width=0.5, color=(255,0,255,0.2))

# p.line(data_input_flat, data_target_flat, line_width=0.5, color=(0,0,255,0.5))

p.line(test_set_flat, nn_preds.flatten(), line_width=2, color="orange")
# Loss
# p.line(range(epochs), epoch_loss, line_width=2, color="green")

show(p)

In [41]:
loss_p = figure(width=500, height=500)

# Loss
# loss_p.line(range(len(epoch_loss)), epoch_loss, line_width=0.5, color="green")

loss_p.line(range(len(kl_loss)), kl_loss, line_width=0.5, color="orange")

loss_p.line(range(len(data_loss)), data_loss, line_width=0.5, color="purple")

# loss_p.line(range(len(nn_epoch_loss)), nn_epoch_loss, line_width=1, color="green")

show(loss_p)

In [42]:
loss_kl = figure(width=500, height=500)

loss_kl.line(range(len(kl_divs)), kl_divs, line_width=0.5, color="red")

show(loss_kl)